<a href="https://colab.research.google.com/github/cauefeder/Finances/blob/master/RSLsignals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime
import yfinance as yf
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")


 Step 1: Download Stock Data


In [ ]:
stock_symbol = "BTC-USD"
start = "2018-01-01"
end = "2025-03-16"

# Download data with auto_adjust=True (default)
df = yf.download(stock_symbol, start, end)

[*********************100%***********************]  1 of 1 completed


 Step 2: Ensure Necessary Columns Exist



In [ ]:
if "Close" not in df.columns:
    raise KeyError("Missing 'Close' column in the downloaded data.")

Step 3: Compute Basic Metrics


In [ ]:
df["Returns"] = df["Close"].pct_change(1)
df["Adj Low"] = df["Low"]
df["Adj High"] = df["High"]
df["Adj Open"] = df["Open"]
df["Vol"] = df["Returns"].rolling(20).std() * np.sqrt(252)


Step 4: Define Target Variables


In [ ]:
p = 5  # Prediction period

# Ensure there are enough rows before shifting
df["Alvo1"] = df["Close"].shift(-p)

# Check if the column exists AND has at least one non-NaN value
if "Alvo1" in df.columns and df["Alvo1"].notna().any():
    df.dropna(subset=["Alvo1"], inplace=True)


Step 5: Generate Classification Targets
 `Alvo2`: Binary classification (1 if price goes up, 0 otherwise)
`Alvo3`: Percentage change over `p` days

In [ ]:
from datetime import datetime
import yfinance as yf
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

# Get the data
stock_symbol = "BTC-USD"  # Cryptocurrency symbol
start = "2018-01-01"
end = "2025-03-16"  # Updated to current date

df1 = yf.download(stock_symbol, start, end)

# Flatten MultiIndex columns if necessary
if isinstance(df1.columns, pd.MultiIndex):
    df1.columns = ['_'.join(col).strip() for col in df1.columns]

# Ensure the data is in the correct format
if isinstance(df1, pd.DataFrame):
    print("Data downloaded successfully.")
else:
    raise ValueError("Failed to download data. Check the stock symbol or internet connection.")

# Extract necessary columns
df1["Close"] = df1[f"Close_{stock_symbol}"] if f"Close_{stock_symbol}" in df1.columns else df1["Close"]
df1["Open"] = df1[f"Open_{stock_symbol}"] if f"Open_{stock_symbol}" in df1.columns else df1["Open"]
df1["High"] = df1[f"High_{stock_symbol}"] if f"High_{stock_symbol}" in df1.columns else df1["High"]
df1["Low"] = df1[f"Low_{stock_symbol}"] if f"Low_{stock_symbol}" in df1.columns else df1["Low"]

# Calculate Moving Averages
df1["MA5"] = df1["Close"].rolling(5).mean()
df1["MA10"] = df1["Close"].rolling(10).mean()
df1["MA20"] = df1["Close"].rolling(20).mean()
df1["MA52"] = df1["Close"].rolling(52).mean()

df1.dropna(inplace=True)  # Drop rows with NaN before using numpy

# Convert to numpy arrays for calculations
close_array = df1["Close"].to_numpy()
ma5_array = df1["MA5"].to_numpy()
ma10_array = df1["MA10"].to_numpy()
ma20_array = df1["MA20"].to_numpy()
ma52_array = df1["MA52"].to_numpy()

# Calculate RSL values using numpy operations
df1["RSL5"] = np.round((close_array / ma5_array - 1) * 100, 4)
df1["RSL10"] = np.round((close_array / ma10_array - 1) * 100, 4)
df1["RSL20"] = np.round((close_array / ma20_array - 1) * 100, 4)
df1["RSL52"] = np.round((close_array / ma52_array - 1) * 100, 4)

# Add buy/sell signals based on RSL values
# Buy signal conditions
df1["RSL_Buy_Signal"] = ((df1["RSL5"] < -2) &
                         (df1["RSL20"] < -5) &
                         (df1["RSL5"] > df1["RSL5"].shift(1)))

# Sell signal conditions
df1["RSL_Sell_Signal"] = ((df1["RSL5"] > 3) &
                          (df1["RSL20"] > 8) &
                          (df1["RSL5"] < df1["RSL5"].shift(1)))

# Create candlestick chart with RSL indicators
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

# Candlestick chart
fig.add_trace(go.Candlestick(x=df1.index, open=df1["Open"], high=df1["High"],
                            low=df1["Low"], close=df1["Close"],
                            name="BTC-USD",
                            increasing_line_color="green", decreasing_line_color="red"),
             row=1, col=1)

# Add buy signals
buy_signals = df1[df1["RSL_Buy_Signal"]]
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals["Low"] * 0.98,
                        mode="markers",
                        marker=dict(symbol="triangle-up", size=15, color="green"),
                        name="Buy Signal"),
             row=1, col=1)

# Add sell signals
sell_signals = df1[df1["RSL_Sell_Signal"]]
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals["High"] * 1.02,
                        mode="markers",
                        marker=dict(symbol="triangle-down", size=15, color="red"),
                        name="Sell Signal"),
             row=1, col=1)

# RSL indicators
fig.add_trace(go.Scatter(x=df1.index, y=df1["RSL5"],
                        name="RSL5", line=dict(color="red")),
             row=2, col=1)

fig.add_trace(go.Scatter(x=df1.index, y=df1["RSL10"],
                        name="RSL10", line=dict(color="blue")),
             row=2, col=1)

fig.add_trace(go.Scatter(x=df1.index, y=df1["RSL20"],
                        name="RSL20", line=dict(color="purple")),
             row=2, col=1)

# Add zero line for RSL reference
fig.add_trace(go.Scatter(x=df1.index, y=[0] * len(df1),
                        name="Zero Line", line=dict(color="black", dash="dash")),
             row=2, col=1)

# Update layout
fig.update_layout(height=800, width=1000,
                 title_text="BTC-USD RSL Analysis with Buy/Sell Signals",
                 font_color="blue",
                 title_font_color="black",
                 xaxis2_title="Time",
                 yaxis_title="Price (USD)",
                 yaxis2_title="RSL (%)",
                 font=dict(size=15, color="Black"))

# Add range selector
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=False),
        type="date"
    )
)

fig.update_layout(hovermode="x unified")

# Display the chart
fig.show()

[*********************100%***********************]  1 of 1 completed


Data downloaded successfully.
